# Deepfake Detection Notebook

## 1. Install Dependencies

In [ ]:
!pip install tensorflow pillow numpy flask pymongo gunicorn

## 2. Download Datasets

In [ ]:
import requests
import os
import time

output_dir = 'image_classification_data/fake'
os.makedirs(output_dir, exist_ok=True)

num_images = 100

url = 'https://thispersondoesnotexist.com/'

for i in range(num_images):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        
        filename = os.path.join(output_dir, f'fake_{i+1}.jpg')

        with open(filename, 'wb') as f:
            f.write(response.content)

        print(f'Successfully downloaded {filename}')

    except requests.exceptions.RequestException as e:
        print(f'Error downloading image {i+1}: {e}')

    time.sleep(1)

print('Finished downloading fake images.')

In [ ]:
import os
from sklearn.datasets import fetch_lfw_people
import matplotlib.pyplot as plt

output_dir = 'image_classification_data/real'
os.makedirs(output_dir, exist_ok=True)

lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

for i, (image, target) in enumerate(zip(lfw_people.images, lfw_people.target)):
    filename = os.path.join(output_dir, f'real_{i}.jpg')
    plt.imsave(filename, image, cmap='gray')

    if (i + 1) % 100 == 0:
        print(f'Saved {i+1} images.')

print(f'Finished downloading and saving {len(lfw_people.images)} real images.')

## 3. Train the Deepfake Detector Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_WIDTH, IMG_HEIGHT = 128, 128
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) 

train_generator = train_datagen.flow_from_directory(
    'image_classification_data',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    'image_classification_data',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation')

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

EPOCHS = 15
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE)

model.save('deepfake_detector_model.h5')

print("Model training complete and saved as deepfake_detector_model.h5")

## 4. Run the Web App (using Flask and ngrok)

In [ ]:
!pip install flask-ngrok

In [ ]:
import os
from flask import Flask, request, jsonify, render_template
from flask_ngrok import run_with_ngrok
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
import numpy as np
import io

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

MODEL_PATH = 'deepfake_detector_model.h5'
try:
    model = load_model(MODEL_PATH)
    print(f"Model '{MODEL_PATH}' loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None

def prepare_image(image, target_size):
    if image.mode != "RGB":
        image = image.convert("RGB")
    image = image.resize(target_size)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    return image

@app.route('/')
def index():
    return '''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Deepfake Detector</title>
    </head>
    <body>
        <h1>Deepfake Detector</h1>
        <form id="upload-form" enctype="multipart/form-data">
            <input type="file" name="file" id="file-input">
            <button type="submit">Predict</button>
        </form>
        <div id="result"></div>
    
        <script>
            document.getElementById('upload-form').addEventListener('submit', async (e) => {
                e.preventDefault();
                const formData = new FormData();
                const fileInput = document.getElementById('file-input');
                formData.append('file', fileInput.files[0]);
    
                const response = await fetch('/predict', {
                    method: 'POST',
                    body: formData
                });
    
                const result = await response.json();
    
                const resultDiv = document.getElementById('result');
                resultDiv.innerHTML = `<p>Prediction: ${result.prediction}</p>`;
            });
        </script>
    </body>
    </html>
    '''

@app.route('/predict', methods=['POST'])
def predict():
    if model is None:
        return jsonify({"error": "Model not loaded"}), 500
    if 'file' not in request.files:
        return jsonify({"error": "No file part"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    try:
        image = Image.open(io.BytesIO(file.read()))
        prepared_image = prepare_image(image, target_size=(128, 128))
        
        prediction = model.predict(prepared_image)
        prediction_label = 'Deepfake' if prediction[0][0] > 0.5 else 'Real'
        
        return jsonify({"prediction": prediction_label})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run()